Correspondence Analysis of alcoholic beverages and where to drink

In [1]:
# Importing libs
import numpy as np
import pandas as pd
from prince import CA
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Importing and processing data

In [2]:

df=pd.read_excel("data\\data.xlsx",sheet_name="data")
""" 
Data from a survey that asked where was the best place (from a list of 11 places) to drink 12 types of alcoolic beverages
 
weight: variable to match the sociodemographic characteristics of the sample to the population
 
Variables with "place" and "drink": each column is a beverage type and it's value is the name of the place if it was select or 0 if it wasnt
Some of the beverages that were in the survey weren't asked in this part so we will have to delete some of the variables
"""
df.head(5)

,ID,weight,place_1_drink_11,place_2_drink_11,place_3_drink_11,place_4_drink_11,place_5_drink_11,place_6_drink_11,place_7_drink_11,place_8_drink_11,...,place_3_drink_23,place_4_drink_23,place_5_drink_23,place_6_drink_23,place_7_drink_23,place_8_drink_23,place_9_drink_23,place_10_drink_23,place_11_drink_23,place_12_drink_23
0,1,0.491357,0,0,0,0,Home,0,0,Pub,...,0,0,0,0,0,0,0,0,0,None
1,2,0.016819,0,0,0,Friend\'s house,Home,0,0,0,...,0,0,Home,0,0,Pub,0,0,0,0
2,3,0.109428,0,Restaurante,0,0,0,0,0,Pub,...,0,0,0,0,0,0,0,Beach,Pool,0
3,4,0.171597,0,0,0,0,0,Show,0,0,...,Grocery Shop,0,0,0,House party,0,0,0,0,0
4,5,0.425068,0,0,0,0,Home,0,0,Pub,...,0,0,Home,0,0,0,Bar,0,Pool,0


In [3]:
places=12 ### number of places +1
drinks=24 ### number of drinks +1

#* Getting variable names based on number of places and drinks
var_names=[]
for iplaces in range(1,places):
    for idrinks in range(11,drinks):
       var_names.append(f"place_{iplaces}_drink_{idrinks}") 

# Making one column with the values of all variables and changing it's value to binary (1 if they like the mix of the drink with the place) to make it easier to work with weights
df2 = pd.melt(df,id_vars=['ID','weight'],value_vars=var_names, value_name=('place_and_drink')).dropna()
df2.loc[df2['place_and_drink'] != 0, 'place_and_drink'] = 1

# Applying weights
df2['place_and_drink_weighted']=df2['place_and_drink']*df2['weight']
# creating variable for place and drink
for iplaces in range(1,places):    
    for idrinks in range(11,drinks):
        df2.loc[df2['variable']==f"place_{iplaces}_drink_{idrinks}",('drink')]=idrinks
        df2.loc[df2['variable']==f"place_{iplaces}_drink_{idrinks}",('place')]=iplaces

In [4]:
# Creating the contingency table
contingency_table = pd.pivot_table(
                  df2, 
                  values='place_and_drink_weighted', 
                  index=['place'],
                  columns=['drink'],
                  aggfunc=np.sum
)
# Labeling
column_names=[
'Beer 1',
'Cheap Spirit',
'Beer 2',
'Flavoured Spirit 4',
'Flavoured Spirit 1',
'Beer 3',
'Flavoured Beer 1',
'Flavoured Beer 2',
'Flavoured Spirit 2',
'Flavoured Spirit 3'
]
row_names=[
'Night club',
'Restaurant',
'Grocery Shop',
'Friend\'s house',
'Home',
'Show',
'House party',
'Pub',
'Bar',
'Beach',
'Pool'
]
contingency_table.columns=column_names
contingency_table.index=row_names

print (contingency_table)

                   Beer 1  Cheap Spirit     Beer 2  Flavoured Spirit 4  \
Night club      26.478609     20.676671  43.091478           47.713263   
Restaurant      17.921890     11.693016  26.819904           18.017244   
Grocery Shop    22.493552     19.364991  15.083082           16.266758   
Friend's house  39.918902     44.001308  42.580234           32.060621   
Home            64.031607     48.650806  61.656867           53.641644   
Show            27.684540     13.963411  27.508136           37.897687   
House party     51.544614     48.517885  55.975748           48.535382   
Pub             35.998318     15.421897  41.774473           47.647354   
Bar             48.289090     65.258624  34.797344           16.246760   
Beach           48.064503     23.881518  45.414532           47.975835   
Pool            34.260425     20.369498  28.312653           37.654504   

                Flavoured Spirit 1     Beer 3  Flavoured Beer 1  \
Night club               52.293465  21.83826

Preparing correspondence analysis

In [5]:
ca=CA(n_components=2,n_iter=10,random_state=18012021)
ca.fit(contingency_table)
ca.row_coordinates(contingency_table)
ca.column_coordinates(contingency_table)
# variation explained by CA.
ca.explained_inertia_


[0.7786441926436779, 0.13732578583501656]

Creating Map

In [6]:
# Creating datasets with the coordinates
coord_x=ca.row_coordinates(contingency_table)
coord_y=ca.column_coordinates(contingency_table)
coord_x.columns=['x','y']
coord_y.columns=['x','y']
# Assigning a series variable to know which coordinate belongs to places/drinks and merging dfs

coord=pd.concat([coord_x.assign(series='place'),coord_y.assign(series='drink')])
plt.figure(figsize=(10,10),dpi=1000,frameon=False)
sns.set_theme(style="whitegrid")
plot = sns.scatterplot(
    x='x',y='y',data=coord,hue='series',s=150,style='series',legend=False,ax=None,palette=sns.color_palette("flare", 2),markers=['o','D']
)

# Adding labels to points
i=-1
for x, y in zip(coord['x'], coord['y']):
    i+=1
    color='black'
    if i>10:    color=sns.color_palette("flare", 2)[1] # Changing drinks colors
    if x!=coord['x'].nlargest(2)[0] and x!=coord['x'].nlargest(2)[1]:
        plt.text(x = x+0.015,y = y-0.0025,s=coord.index[i],weight='bold',color=color)
    elif x==coord['x'].nlargest(2)[0] : # Making all labels fit in the plot
        plt.text(x = x-0.055,y = y-0.0025,s=coord.index[i],weight='bold',color=color)
    else:   plt.text(x = x-0.145,y = y-0.0025,s=coord.index[i],weight='bold',color=color)


# Cleaning axes

plot.set(xticklabels=[])
plot.set(xlabel=None)
plot.set(yticklabels=[])
plot.set(ylabel=None)
# Setting title
plt.title('Association map',weight='bold',size=20)




Text(0.5, 1.0, 'Association map')